In [43]:
import pandas as pd
from collections import Counter

In [469]:
pd.set_option('display.max_rows', 5)

# Utils

In [45]:
def transform_df_to_budget_format(df, af_name, model_name):
    """
    Transforms the input df into the required format.
    """
    model_df = df[df["Model_name"] == model_name].sort_values(["Dataset", "AcqusitionFunction", "Budget"])
    df_af_type = model_df[model_df["AcqusitionFunction"] == af_name]
    df_acq_fun = df_af_type.loc[:, ['Dataset', 'Budget', 'F1-score']]
    df_pivot = df_acq_fun.pivot(index="Dataset", columns="Budget", values="F1-score").reset_index()
    df_pivot.columns = ["Dataset", "budget1", "budget2", "budget3"]

    return df_pivot

def dataframe_to_latex(df):
    latex_str = ""
    for _, row in df.iterrows():
        dataset_name = row["Dataset"].replace("_", r"\_")  # Escape underscores
        row_values = [dataset_name] + [f"{val:.3f}" for val in row.iloc[1:]]
        latex_str += " & ".join(row_values) + " \\\\\n        \\hline\n"
    return latex_str

# Load Data

In [46]:
df_basic_AL = pd.read_csv("/Users/marysia/Desktop/Repo/ActiveLearning/FINALL_RESULTS/ALL_BASIC_RESULTS.csv")
df_custom_AL = pd.read_csv("/Users/marysia/Desktop/Repo/ActiveLearning/FINALL_RESULTS/ALL_CUSTOM_RESULTS.csv")

# Basic AL

In [470]:
df_basic_AL.head()

,Unnamed: 0,Cycle,Class,Precision,Recall,F1-score,Accuracy,Labelled Samples,Model_name,Dataset,AcqusitionFunction,Budget
0,2.0,2,1.0,0.836952,0.520833,0.624190,0.933973,148.0,KNN,yeast3,least_confidence,30
1,2.0,2,1.0,0.336404,0.363684,0.329999,0.881353,110.0,KNN,yeast-0-2-5-6_vs_3-7-8-9,least_confidence,30
2,2.0,2,1.0,0.931429,0.885000,0.897141,0.993314,209.0,KNN,kr-vs-k-one_vs_fifteen,least_confidence,30
3,2.0,2,1.0,0.731300,0.489913,0.564371,0.926534,467.0,KNN,page-blocks0,least_confidence,30
4,2.0,2,1.0,0.000000,0.000000,0.000000,0.901345,43.0,KNN,glass-0-1-5_vs_2,least_confidence,30


In [471]:
df_basic_AL.sort_values(['Dataset', 'Model_name', 'Budget', 'AcqusitionFunction']).head()

,Unnamed: 0,Cycle,Class,Precision,Recall,F1-score,Accuracy,Labelled Samples,Model_name,Dataset,AcqusitionFunction,Budget
910,NaN,2,1.0,0.000000,0.000000,0.000000,0.968771,217.0,KNN,abalone-17_vs_7-8-9-10,entropy,30
73,2.0,2,1.0,0.003125,0.018182,0.005333,0.941374,217.0,KNN,abalone-17_vs_7-8-9-10,least_confidence,30
1747,NaN,2,1.0,0.000000,0.000000,0.000000,0.974337,217.0,KNN,abalone-17_vs_7-8-9-10,margin_sampling,30
2584,NaN,2,1.0,0.013333,0.018182,0.015385,0.969198,217.0,KNN,abalone-17_vs_7-8-9-10,random_sampling,30
1189,NaN,3,1.0,0.023169,0.072727,0.032051,0.943935,227.0,KNN,abalone-17_vs_7-8-9-10,entropy,40


In [472]:
knn_df = df_basic_AL[df_basic_AL["Model_name"] == "KNN"].sort_values(["Dataset", "AcqusitionFunction", "Budget"])
#knn_df_marg_sam = knn_df[knn_df["AcqusitionFunction"] == "entropy"]
# knn_df_1 = knn_df_marg_sam.loc[:, ['Dataset', 'Budget', 'F1-score']]
# knn_df_1

In [473]:
for column in ["AcqusitionFunction", "Model_name"]:
    print(f"Unique values in {column}: {pd.unique(df_basic_AL[column])}")

Unique values in AcqusitionFunction: ['least_confidence' 'entropy' 'margin_sampling' 'random_sampling']
Unique values in Model_name: ['KNN' 'decision_tree' 'LogisticRegression']


In [474]:
df_basic_AL[df_basic_AL["Dataset"] == "dermatology-6"].sort_values(["Model_name", "AcqusitionFunction", "Budget"])

,Unnamed: 0,Cycle,Class,Precision,Recall,F1-score,Accuracy,Labelled Samples,Model_name,Dataset,AcqusitionFunction,Budget
844,NaN,2,1.0,0.457111,0.539286,0.372376,0.792135,65.0,KNN,dermatology-6,entropy,30
1123,NaN,3,1.0,0.613411,0.660714,0.485556,0.834831,75.0,KNN,dermatology-6,entropy,40
...,...,...,...,...,...,...,...,...,...,...,...,...
2890,NaN,3,1.0,0.788889,0.950000,0.843077,0.974804,68.0,decision_tree,dermatology-6,random_sampling,40
3169,NaN,4,1.0,0.760000,0.900000,0.805714,0.966354,78.0,decision_tree,dermatology-6,random_sampling,50


In [475]:
# models = ['KNN', 'decision_tree', 'LogisticRegression']
acquistion_functions = [ 'entropy', 'margin_sampling', 'least_confidence']

knn_transformed_df_basic_AL = pd.DataFrame()

model = 'KNN'
# for model in models:
for acq_func in acquistion_functions:
    print(model, acq_func)
    specific_df = transform_df_to_budget_format(df_basic_AL, acq_func, model)
    specific_df = specific_df.rename(columns={col: f"{col}_{acq_func}" for col in specific_df.columns if col != "Dataset"})

    if knn_transformed_df_basic_AL.empty == True:
        # specific_df = specific_df.rename(columns={col: f"{col}_{acq_func}" for col in specific_df.columns if col != "Dataset"})
        knn_transformed_df_basic_AL = specific_df
    else:
        knn_transformed_df_basic_AL = pd.merge(knn_transformed_df_basic_AL, specific_df, on='Dataset', how='outer')

KNN entropy
KNN margin_sampling
KNN least_confidence


In [476]:
models = ['KNN', 'decision_tree', 'LogisticRegression']
acquisition_functions = ['entropy', 'margin_sampling', 'least_confidence', 'random_sampling']

# Transformed DataFrames for each model (key -> model, value -> df)
transformed_dfs = {}

for model in models:    
    transformed_df = pd.DataFrame() 

    for acq_func in acquisition_functions:
        specific_df = transform_df_to_budget_format(df_basic_AL, acq_func, model)
        specific_df = specific_df.rename(columns={col: f"{col}_{acq_func}" for col in specific_df.columns if col != "Dataset"})

        if transformed_df.empty:
            transformed_df = specific_df  
        else:
            transformed_df = pd.merge(transformed_df, specific_df, on='Dataset', how='inner')

    transformed_dfs[model] = transformed_df

In [477]:
transformed_dfs['KNN'].head()

,Dataset,budget1_entropy,budget2_entropy,budget3_entropy,budget1_margin_sampling,budget2_margin_sampling,budget3_margin_sampling,budget1_least_confidence,budget2_least_confidence,budget3_least_confidence,budget1_random_sampling,budget2_random_sampling,budget3_random_sampling
0,abalone-17_vs_7-8-9-10,0.000000,0.032051,0.038363,0.00,0.027273,0.034286,0.005333,0.011765,0.041053,0.015385,0.098071,0.030769
1,abalone-19_vs_10-11-12-13,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,abalone-20_vs_8-9-10,0.000000,0.000000,0.028571,0.04,0.057143,0.000000,0.000000,0.000000,0.069697,0.000000,0.050000,0.000000
3,abalone-21_vs_8,0.000000,0.160000,0.000000,0.00,0.160000,0.160000,0.260000,0.114286,0.160000,0.100000,0.000000,0.000000
4,abalone-3_vs_11,0.931429,0.860000,0.860000,0.86,0.831429,0.860000,0.860000,0.833333,0.804762,0.900000,0.860000,0.860000


In [478]:
knn_filtered = transformed_dfs['KNN'].loc[~(transformed_dfs['KNN'].iloc[:, 1:] == 0.0).all(axis=1)]
lg_filtered = transformed_dfs['LogisticRegression'].loc[~(transformed_dfs['LogisticRegression'].iloc[:, 1:] == 0.0).all(axis=1)]
dt_filtered = transformed_dfs['decision_tree'].loc[~(transformed_dfs['decision_tree'].iloc[:, 1:] == 0.0).all(axis=1)]

In [479]:
knn_filtered_sorted = knn_filtered.sort_values(['budget3_entropy', 'budget1_entropy', 'budget2_entropy'], ascending=False)
lg_filtered_sorted = lg_filtered.sort_values(['budget3_entropy', 'budget1_entropy', 'budget2_entropy'], ascending=False)
dt_filtered_sorted = dt_filtered.sort_values(['budget3_entropy', 'budget1_entropy', 'budget2_entropy'], ascending=False)

# Custom AL

In [484]:
df_custom_AL.sort_values(['Dataset', 'Model_name', 'AcqusitionFunction'])[:10]#.mean(numeric_only=True)

,Unnamed: 0,Cycle,Class,Precision,Recall,F1-score,Accuracy,Labelled Samples,Model_name,Dataset,AcqusitionFunction,Budget
910,NaN,4,1.0,0.0,0.0,0.0,0.975194,237.0,KNN,abalone-17_vs_7-8-9-10,entropy,30
1189,NaN,4,1.0,0.0,0.0,0.0,0.975194,237.0,KNN,abalone-17_vs_7-8-9-10,entropy,40
...,...,...,...,...,...,...,...,...,...,...,...,...
2305,NaN,4,1.0,0.0,0.0,0.0,0.975194,237.0,KNN,abalone-17_vs_7-8-9-10,margin_sampling,50
2584,NaN,4,1.0,0.0,0.0,0.0,0.974337,237.0,KNN,abalone-17_vs_7-8-9-10,random_sampling,30


In [485]:
transformed_customs_dfs = {}

for model in models:
    print(f"Processing model: {model}")
    
    transformed_custom_df = pd.DataFrame()  # Initialize an empty DataFrame for this model

    for acq_func in acquisition_functions:
        specific_df = transform_df_to_budget_format(df_custom_AL, acq_func, model)
        specific_df = specific_df.rename(columns={col: f"{col}_{acq_func}" for col in specific_df.columns if col != "Dataset"})

        if transformed_custom_df.empty:
            transformed_custom_df = specific_df  
        else:
            transformed_custom_df = pd.merge(transformed_custom_df, specific_df, on='Dataset', how='inner')

    transformed_customs_dfs[model] = transformed_custom_df

Processing model: KNN
Processing model: decision_tree
Processing model: LogisticRegression


In [486]:
knn_filtered_cm = transformed_customs_dfs['KNN'].loc[~(transformed_customs_dfs['KNN'].iloc[:, 1:] == 0.0).all(axis=1)]
lg_filtered_cm = transformed_customs_dfs['LogisticRegression'].loc[~(transformed_customs_dfs['LogisticRegression'].iloc[:, 1:] == 0.0).all(axis=1)]
dt_filtered_cm = transformed_customs_dfs['decision_tree'].loc[~(transformed_customs_dfs['decision_tree'].iloc[:, 1:] == 0.0).all(axis=1)]

In [487]:
knn_filtered_cm_sorted = knn_filtered_cm.sort_values(['budget3_entropy', 'budget1_entropy', 'budget2_entropy'], ascending=False)
lg_filtered_cm_sorted = lg_filtered_cm.sort_values(['budget3_entropy', 'budget1_entropy', 'budget2_entropy'], ascending=False)
dt_filtered_cm_sorted = dt_filtered_cm.sort_values(['budget3_entropy', 'budget1_entropy', 'budget2_entropy'], ascending=False)

In [488]:
dt_filtered_cm

,Dataset,budget1_entropy,budget2_entropy,budget3_entropy,budget1_margin_sampling,budget2_margin_sampling,budget3_margin_sampling,budget1_least_confidence,budget2_least_confidence,budget3_least_confidence,budget1_random_sampling,budget2_random_sampling,budget3_random_sampling
0,abalone-17_vs_7-8-9-10,0.077922,0.000000,0.000000,0.333333,0.058480,0.152778,0.000000,0.000000,0.054054,0.000000,0.000000,0.000000
1,abalone-19_vs_10-11-12-13,0.021739,0.000000,0.000000,0.052632,0.000000,0.185185,0.370370,0.185185,0.424693,0.047059,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,yeast5,0.545455,0.267532,0.522727,0.615385,0.625000,0.500000,0.173913,0.336957,0.319876,0.100000,0.392365,0.183333
92,yeast6,0.312500,0.211806,0.286131,0.375000,0.330357,0.220238,0.476190,0.238095,0.206349,0.000000,0.000000,0.051282


# Comparing the results

In [68]:
merged_df = df_basic_AL.merge(df_custom_AL, on=["Budget", "Model_name", "Dataset", "AcqusitionFunction"], suffixes=("_basic", "_custom"), how='inner')

In [69]:
merged_df["Higher_F1_in"] = merged_df.apply(lambda row: "Basic AL" if row["F1-score_basic"] > row["F1-score_custom"] else ("Custom AL" if row["F1-score_basic"] < row["F1-score_custom"] else "Equal"), axis=1)

In [70]:
merged_df.head()

,Unnamed: 0_basic,Cycle_basic,Class_basic,Precision_basic,Recall_basic,F1-score_basic,Accuracy_basic,Labelled Samples_basic,Model_name,Dataset,...,Budget,Unnamed: 0_custom,Cycle_custom,Class_custom,Precision_custom,Recall_custom,F1-score_custom,Accuracy_custom,Labelled Samples_custom,Higher_F1_in
0,2.0,2,1.0,0.869167,0.399242,0.527663,0.925216,148.0,KNN,yeast3,...,30,4.0,4,1.0,0.646991,0.644508,0.632278,0.913079,168.0,Custom AL
1,2.0,2,1.0,0.506893,0.200526,0.262501,0.908338,110.0,KNN,yeast-0-2-5-6_vs_3-7-8-9,...,30,4.0,4,1.0,0.689231,0.290000,0.380672,0.922294,130.0,Custom AL
2,2.0,2,1.0,0.945455,0.831667,0.865371,0.991531,209.0,KNN,kr-vs-k-one_vs_fifteen,...,30,4.0,4,1.0,1.000000,1.000000,1.000000,1.000000,229.0,Custom AL
3,2.0,2,1.0,0.793298,0.498826,0.597673,0.933112,467.0,KNN,page-blocks0,...,30,4.0,4,1.0,0.692471,0.532706,0.569814,0.922514,487.0,Basic AL
4,2.0,2,1.0,0.000000,0.000000,0.000000,0.889580,43.0,KNN,glass-0-1-5_vs_2,...,30,4.0,4,1.0,0.000000,0.000000,0.000000,0.901345,63.0,Equal


In [71]:
filtered_df = merged_df[(merged_df["F1-score_basic"] != 0) & (merged_df["F1-score_custom"] != 0)]

In [72]:
Counter(filtered_df["Higher_F1_in"])

Counter({'Custom AL': 877, 'Basic AL': 699, 'Equal': 8})

In [73]:
df_custom_AL.sort_values(['Model_name', 'AcqusitionFunction']).mean(numeric_only=True)

Unnamed: 0            4.000000
Cycle                 4.000000
Class                 1.000000
Precision             0.253534
Recall                0.265295
F1-score              0.227425
Accuracy              0.891230
Labelled Samples    138.792055
Budget               40.000000
dtype: float64

# Statistical tests

In [74]:
from scipy.stats import wilcoxon

## Classical AL vs Weighted AL

### For KNN

In [75]:
basic_knn =  transformed_dfs['KNN']
custom_knn = transformed_customs_dfs['KNN']

In [76]:
basic_knn.mean(numeric_only=True)

budget1_entropy             0.179681
budget2_entropy             0.189046
budget3_entropy             0.198815
budget1_margin_sampling     0.198734
budget2_margin_sampling     0.196876
budget3_margin_sampling     0.196531
budget1_least_confidence    0.187544
budget2_least_confidence    0.191967
budget3_least_confidence    0.201749
budget1_random_sampling     0.194602
budget2_random_sampling     0.204212
budget3_random_sampling     0.216302
dtype: float64

In [77]:
custom_knn.mean(numeric_only=True)

budget1_entropy             0.190511
budget2_entropy             0.204612
budget3_entropy             0.209424
budget1_margin_sampling     0.223150
budget2_margin_sampling     0.215477
budget3_margin_sampling     0.203690
budget1_least_confidence    0.242162
budget2_least_confidence    0.223833
budget3_least_confidence    0.220226
budget1_random_sampling     0.214420
budget2_random_sampling     0.211292
budget3_random_sampling     0.212058
dtype: float64

In [78]:
columns = basic_knn.columns
columns = columns.drop('Dataset')

results = []

for col in columns:
    stat, p = wilcoxon(basic_knn[col], custom_knn[col])
    results.append([col, stat, p, p<0.05])

results_df = pd.DataFrame(results, columns=["Budget Column", "Wilcoxon Statistic", "P-Value", "Statistical Significant"])

In [79]:
results_df

,Budget Column,Wilcoxon Statistic,P-Value,Statistical Significant
0,budget1_entropy,633.5,0.057637,False
1,budget2_entropy,976.0,0.083542,False
2,budget3_entropy,928.0,0.030295,True
3,budget1_margin_sampling,817.0,0.094962,False
4,budget2_margin_sampling,884.5,0.024146,True
5,budget3_margin_sampling,1158.0,0.289911,False
6,budget1_least_confidence,290.0,0.000004,True
7,budget2_least_confidence,865.0,0.011743,True
8,budget3_least_confidence,856.0,0.052737,False
9,budget1_random_sampling,783.0,0.026152,True


In [80]:
latex_table = results_df.to_latex(index=False, escape=False)

/var/folders/bl/bmspcss576x1_nsjjqdhbjvw0000gn/T/ipykernel_91345/2675711920.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = results_df.to_latex(index=False, escape=False)


In [81]:
print(latex_table)

\begin{tabular}{lrrl}
\toprule
           Budget Column &  Wilcoxon Statistic &  P-Value &  Statistical Significant \\
\midrule
         budget1_entropy &               633.5 & 0.057637 &                    False \\
         budget2_entropy &               976.0 & 0.083542 &                    False \\
         budget3_entropy &               928.0 & 0.030295 &                     True \\
 budget1_margin_sampling &               817.0 & 0.094962 &                    False \\
 budget2_margin_sampling &               884.5 & 0.024146 &                     True \\
 budget3_margin_sampling &              1158.0 & 0.289911 &                    False \\
budget1_least_confidence &               290.0 & 0.000004 &                     True \\
budget2_least_confidence &               865.0 & 0.011743 &                     True \\
budget3_least_confidence &               856.0 & 0.052737 &                    False \\
 budget1_random_sampling &               783.0 & 0.026152 &                     

### For Logistic Regression

In [82]:
basic_lg =  transformed_dfs['LogisticRegression']
custom_lg = transformed_customs_dfs['LogisticRegression']

In [83]:
basic_lg.mean(numeric_only=True)

budget1_entropy             0.087779
budget2_entropy             0.087613
budget3_entropy             0.083857
budget1_margin_sampling     0.090208
budget2_margin_sampling     0.082364
budget3_margin_sampling     0.088967
budget1_least_confidence    0.093203
budget2_least_confidence    0.086417
budget3_least_confidence    0.083869
budget1_random_sampling     0.092152
budget2_random_sampling     0.089142
budget3_random_sampling     0.089483
dtype: float64

In [84]:
custom_lg.mean(numeric_only=True)

budget1_entropy             0.095076
budget2_entropy             0.092595
budget3_entropy             0.096220
budget1_margin_sampling     0.098470
budget2_margin_sampling     0.101623
budget3_margin_sampling     0.101483
budget1_least_confidence    0.090267
budget2_least_confidence    0.105209
budget3_least_confidence    0.098372
budget1_random_sampling     0.111595
budget2_random_sampling     0.106135
budget3_random_sampling     0.097330
dtype: float64

In [85]:
columns = basic_lg.columns
columns = columns.drop('Dataset')

results = []

for col in columns:
    stat, p = wilcoxon(basic_lg[col], custom_lg[col])
    results.append([col, stat, p, p<0.05])

results_df_lg = pd.DataFrame(results, columns=["Budget Column", "Wilcoxon Statistic", "P-Value", "Statistical Significant"])

In [86]:
results_df_lg

,Budget Column,Wilcoxon Statistic,P-Value,Statistical Significant
0,budget1_entropy,148.0,0.324612,False
1,budget2_entropy,146.0,0.301569,False
2,budget3_entropy,127.0,0.083518,False
3,budget1_margin_sampling,124.0,0.190875,False
4,budget2_margin_sampling,112.0,0.038247,True
5,budget3_margin_sampling,155.0,0.176551,False
6,budget1_least_confidence,187.0,0.509557,False
7,budget2_least_confidence,146.0,0.122079,False
8,budget3_least_confidence,165.0,0.103832,False
9,budget1_random_sampling,100.0,0.055168,False


### For DecisionTree

In [87]:
basic_dt =  transformed_dfs['decision_tree']
custom_dt = transformed_customs_dfs['decision_tree']

In [88]:
columns = basic_dt.columns
columns = columns.drop('Dataset')

results = []

for col in columns:
    stat, p = wilcoxon(basic_dt[col], custom_dt[col])
    results.append([col, stat, p, p<0.05])

results_df_dt = pd.DataFrame(results, columns=["Budget Column", "Wilcoxon Statistic", "P-Value", "Statistical Significant"])

In [89]:
results_df_dt

,Budget Column,Wilcoxon Statistic,P-Value,Statistical Significant
0,budget1_entropy,1959.0,0.721768,False
1,budget2_entropy,885.0,0.000001,True
2,budget3_entropy,1624.0,0.044925,True
3,budget1_margin_sampling,1968.0,0.749057,False
4,budget2_margin_sampling,2016.0,0.760550,False
5,budget3_margin_sampling,1807.0,0.333196,False
6,budget1_least_confidence,1938.0,0.433819,False
7,budget2_least_confidence,1726.0,0.078303,False
8,budget3_least_confidence,1716.0,0.072029,False
9,budget1_random_sampling,1331.0,0.001060,True


In [97]:
basic_dt.mean(numeric_only=True)

budget1_entropy             0.431111
budget2_entropy             0.437047
budget3_entropy             0.430526
budget1_margin_sampling     0.426238
budget2_margin_sampling     0.428342
budget3_margin_sampling     0.441754
budget1_least_confidence    0.421868
budget2_least_confidence    0.423531
budget3_least_confidence    0.440344
budget1_random_sampling     0.437631
budget2_random_sampling     0.426978
budget3_random_sampling     0.425612
dtype: float64

In [98]:
custom_dt.mean(numeric_only=True)

budget1_entropy             0.408390
budget2_entropy             0.255631
budget3_entropy             0.351482
budget1_margin_sampling     0.439147
budget2_margin_sampling     0.430377
budget3_margin_sampling     0.407433
budget1_least_confidence    0.392933
budget2_least_confidence    0.349986
budget3_least_confidence    0.382417
budget1_random_sampling     0.317776
budget2_random_sampling     0.351336
budget3_random_sampling     0.335145
dtype: float64

## Acquistion function comparison

In [90]:
knn_filtered = transformed_dfs['KNN'].loc[~(transformed_dfs['KNN'].iloc[:, 1:] == 0.0).all(axis=1)]
lg_filtered = transformed_dfs['LogisticRegression'].loc[~(transformed_dfs['LogisticRegression'].iloc[:, 1:] == 0.0).all(axis=1)]
dt_filtered = transformed_dfs['decision_tree'].loc[~(transformed_dfs['decision_tree'].iloc[:, 1:] == 0.0).all(axis=1)]

In [91]:
import scipy.stats as stats

budgets = {
    "budget1": ["budget1_entropy", "budget1_margin_sampling", "budget1_least_confidence", "budget1_random_sampling"],
    "budget2": ["budget2_entropy", "budget2_margin_sampling", "budget2_least_confidence", "budget2_random_sampling"],
    "budget3": ["budget3_entropy", "budget3_margin_sampling", "budget3_least_confidence", "budget3_random_sampling"]
}

# KNN
friedman_results = {}
for budget, cols in budgets.items():
    stat, p_value = stats.friedmanchisquare(*transformed_customs_dfs['KNN'][cols].values.T)
    friedman_results[budget] = {"Friedman Statistic": stat, "p-value": p_value}

for budget, result in friedman_results.items():
    print(f"Results for {budget}:")
    print(result)
    print('--------')

Results for budget1:
{'Friedman Statistic': 3.978369384359301, 'p-value': 0.26380933861847744}
--------
Results for budget2:
{'Friedman Statistic': 2.8009708737862553, 'p-value': 0.4233401188804621}
--------
Results for budget3:
{'Friedman Statistic': 5.054400000000139, 'p-value': 0.16785677461026016}
--------


In [92]:
friedman_results = {}
for budget, cols in budgets.items():
    stat, p_value = stats.friedmanchisquare(*transformed_customs_dfs['LogisticRegression'][cols].values.T)
    friedman_results[budget] = {"Friedman Statistic": stat, "p-value": p_value}

for budget, result in friedman_results.items():
    print(f"Results for {budget}:")
    print(result)
    print('--------')

Results for budget1:
{'Friedman Statistic': 2.55681818181826, 'p-value': 0.46511021599590696}
--------
Results for budget2:
{'Friedman Statistic': 1.2359154929581113, 'p-value': 0.74440380249075}
--------
Results for budget3:
{'Friedman Statistic': 2.3377926421404633, 'p-value': 0.5053195268573152}
--------


In [93]:
friedman_results = {}
for budget, cols in budgets.items():
    stat, p_value = stats.friedmanchisquare(*transformed_customs_dfs['decision_tree'][cols].values.T)
    friedman_results[budget] = {"Friedman Statistic": stat, "p-value": p_value}

for budget, result in friedman_results.items():
    print(f"Results for {budget}:")
    print(result)
    print('--------')

Results for budget1:
{'Friedman Statistic': 6.754512635379025, 'p-value': 0.08014783298367782}
--------
Results for budget2:
{'Friedman Statistic': 22.777909738717216, 'p-value': 4.4924341666365514e-05}
--------
Results for budget3:
{'Friedman Statistic': 5.976136363636455, 'p-value': 0.11277703260641785}
--------


# Post-hoc test

In [94]:
from itertools import combinations

df = transformed_customs_dfs['decision_tree']
# Select budget1 acquisition functions
budget1_acq_funcs = ["budget1_entropy", "budget1_margin_sampling", "budget1_least_confidence", "budget1_random_sampling"]

# Perform pairwise Wilcoxon Signed-Rank tests
wilcoxon_results = {}
for func1, func2 in combinations(budget1_acq_funcs, 2):
    try:
        stat, p_value = stats.wilcoxon(df[func1], df[func2])
        wilcoxon_results[f"{func1} vs {func2}"] = p_value
    except ValueError:
        wilcoxon_results[f"{func1} vs {func2}"] = "Error (not enough non-zero differences)"

# Convert results into a DataFrame and display
wilcoxon_df = pd.DataFrame.from_dict(wilcoxon_results, orient="index", columns=["p-value"])
print("\nWilcoxon Signed-Rank Test Results (Pairwise Comparisons for Budget1):")
wilcoxon_df


Wilcoxon Signed-Rank Test Results (Pairwise Comparisons for Budget1):


,p-value
budget1_entropy vs budget1_margin_sampling,0.792052
budget1_entropy vs budget1_least_confidence,0.521566
budget1_entropy vs budget1_random_sampling,0.032191
budget1_margin_sampling vs budget1_least_confidence,0.557334
budget1_margin_sampling vs budget1_random_sampling,0.010410
budget1_least_confidence vs budget1_random_sampling,0.087752


In [95]:
budget1_means = df[budget1_acq_funcs].mean()
budget1_means

budget1_entropy             0.408390
budget1_margin_sampling     0.439147
budget1_least_confidence    0.392933
budget1_random_sampling     0.317776
dtype: float64

In [425]:
df = transformed_customs_dfs['KNN']
# Select budget1 acquisition functions
budget1_acq_funcs = ["budget1_entropy", "budget1_margin_sampling", "budget1_least_confidence", "budget1_random_sampling"]

# Perform pairwise Wilcoxon Signed-Rank tests
wilcoxon_results = {}
for func1, func2 in combinations(budget1_acq_funcs, 2):
    try:
        stat, p_value = stats.wilcoxon(df[func1], df[func2])
        wilcoxon_results[f"{func1} vs {func2}"] = p_value
    except ValueError:
        wilcoxon_results[f"{func1} vs {func2}"] = "Error (not enough non-zero differences)"

# Convert results into a DataFrame and display
wilcoxon_df = pd.DataFrame.from_dict(wilcoxon_results, orient="index", columns=["p-value"])
print("\nWilcoxon Signed-Rank Test Results (Pairwise Comparisons for Budget1):")
wilcoxon_df


Wilcoxon Signed-Rank Test Results (Pairwise Comparisons for Budget1):


,p-value
budget1_entropy vs budget1_margin_sampling,0.080512
budget1_entropy vs budget1_least_confidence,0.204041
budget1_entropy vs budget1_random_sampling,0.197004
budget1_margin_sampling vs budget1_least_confidence,0.206326
budget1_margin_sampling vs budget1_random_sampling,0.010829
budget1_least_confidence vs budget1_random_sampling,0.093212


In [427]:
budget1_means = df[budget1_acq_funcs].mean()
budget1_means

budget1_entropy             0.122575
budget1_margin_sampling     0.143764
budget1_least_confidence    0.131850
budget1_random_sampling     0.126323
dtype: float64

budget1_margin_sampling vs budget1_random_sampling	